In [ ]:
def test_import_image_ai():
    HOST = '127.0.0.1'  # The server's hostname or IP address

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        theSbAccess = SBAccess(s)
        theInpSlideId = theSbAccess.GetCurrentSlideId();

        theCapture = 0;
        theNumRows = theSbAccess.GetNumYRows(theCapture)
        theNumColumns = theSbAccess.GetNumXColumns(theCapture)
        theNumPlanes = theSbAccess.GetNumZPlanes(theCapture)

        the3DVolume = np.empty(0,np.uint16)
    
        for theZPlane in range(theNumPlanes):
            image = theSbAccess.ReadImagePlaneBuf(theCapture,0,0,theZPlane,0) #captureid,position,timepoint,zplane,channel
            the3DVolume = np.append(the3DVolume,image)
    
        the3DVolume = the3DVolume.reshape(theNumPlanes,theNumRows, theNumColumns)
        print(the3DVolume.shape)

         # Save the the3DVolume to a 16-bit TIFF file
        file_path = 'original.tif'
        tiff.imwrite(file_path, the3DVolume)

        # Convert to float
        the3DVolume = img_as_float(the3DVolume)  
        # image_uint16 = (image_float * 65535).astype(np.uint16)
  

        # Load a pre-trained StarDist3D model
        model = StarDist3D.from_pretrained('3D_demo')

        # Set the thresholds
        # prob_thresh: The probability threshold for accepting a predicted object. A lower value makes the model more 
        # sensitive, potentially detecting more objects but also increasing false positives.
        # nms_thresh: The non-maximum suppression threshold for suppressing overlapping objects. 
        # A lower value makes the model more strict about overlapping detections, potentially reducing the number 
        # of false positives but also missing closely packed objects.

        prob_thresh = 0.5  # Example probability threshold
        nms_thresh = 0.3   # Example non-maximum suppression threshold

        # Predict instances in the 3D image with custom thresholds
        labels, details = model.predict_instances(the3DVolume, prob_thresh=prob_thresh, nms_thresh=nms_thresh)

        # Visualize the results
        # Here we visualize a middle slice of the 3D volume
        mid_slice = the3DVolume.shape[0] // 2

        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.imshow(the3DVolume[mid_slice], cmap='gray')
        plt.title('Original Image Slice')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(labels[mid_slice], cmap='magma')
        plt.title('Predicted Instances Slice')
        plt.axis('off')

        plt.show()

        # Save the labels to a 16-bit TIFF file
        labels_16 = labels.astype(np.uint16)
        file_path = 'stardist3D.tif'
        tiff.imwrite(file_path, labels_16)

        print(labels_16.size)

        labels_16 = labels_16.reshape(theNumPlanes,theNumRows, theNumColumns)
        print(labels_16.shape)

        print(f"Final Image saved to {file_path}")

        # Add result as new channel
        theNumChannels = theSbAccess.GetNumChannels(theCapture)
        theNewChannel = theNumChannels;


        for theZPlane in range(theNumPlanes):
            image = labels_16[theZPlane, :, :]
            theSbAccess.WriteImagePlaneBuf(theCapture,0,theZPlane,theNewChannel,image)
            theSbAccess.WriteMaskPlaneBuf(theCapture,"AI Nuclei",0,theZPlane,image)

        return